In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
cwd = os.getcwd()
file_path = os.path.join(cwd, 'raw_data.txt')
major = pd.read_csv(r'label_file.csv')
student = pd.read_csv(r'random_student.csv')
student = student.drop_duplicates(['student_id'])

In [3]:
with open(file_path, encoding='utf-8') as file:
    data = file.read()
    data = data.replace('\n', '')
    data = data.split('조회')

In [4]:
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
def time_processing(time):
    bar_idx = time.find('-')
    start_time = time[:bar_idx]
    if time[bar_idx + 1] not in numbers:
        end_time = np.NaN
    else:
        end_time = time[bar_idx + 1:]
    return start_time, end_time

def class_code_processing(class_code):
    new_class_code = class_code
    grade = []
    if new_class_code[0] in numbers:
        while new_class_code[0] in numbers:
            grade.append(new_class_code[0])
            new_class_code = new_class_code[1:]

    return new_class_code, grade

def list_to_string(str_list):
    if len(str_list) == 0:
        return np.NaN
    ret_string = ""
    for ele in str_list:
        ret_string += ele + ', '
        
    ret_string = ret_string.rstrip(', ')
    return ret_string

def class_code_label(class_code):
    label = ""
    for c in class_code:
        if c in numbers:
            return label
        else:
            label += c
    return label

def line_processing(line):
    days = ['월', '화', '수', '목', '금', '토', '일']
    class_types = ['전공기초(11)', '전공선택(05)', '전공필수(04)', '교직전선(20)', '기초교과(16)', '중핵교과(14)', '배분이수(15)', '자유이수(17)', '자유선택(08)', '교직과(06)']
    ret = {}
    ret['class_code'] = np.NaN
    ret['class_name'] = np.NaN
    ret['professor_name'] = np.NaN
    ret['date'] = []
    ret['start_time'] = []
    ret['end_time'] = []
    ret['class_type'] = np.NaN
    ret['grade'] = np.NaN
    ret['label'] = np.NaN
    
    commaidx = line.find(',')
    slashidx = line.find('/')
    for c_type in class_types:
        typeidx = line.find(c_type)
        if typeidx != -1:
            ret['class_type'] = c_type
            break
    if typeidx == -1:
        ret['class_type'] = np.NaN
    idxs = list(filter(lambda x: line[x] == '-', range(len(line))))    # indexes of '-'
    bar_idx = idxs[1]
    for idx in idxs:
        if (idx >= 7 and line[idx - 7] in days) or (idx >= 2 and line[idx - 2] in days):
            bar_idx = idx
            break

    for i, idx in enumerate(idxs):
        if i == 0: #Class_code
            if line[idx + 1] == 'G': #G00
                ret['class_code'] = line[:idx + 4]
                if commaidx != -1 and commaidx < bar_idx:
                    if slashidx == -1 or slashidx >= bar_idx - 7:
                        ret['class_name'] = line[idx + 4:commaidx]
                        ret['professor_name'] = line[commaidx + 1:bar_idx - 8]  
                    elif slashidx < bar_idx - 7:
                        ret['class_name'] = line[idx + 4:commaidx]
                        if line[slashidx + 2] != '.':
                            if bar_idx - 6 != slashidx:
                                ret['professor_name'] = line[commaidx + 1:bar_idx - 8].replace(' / ', ', ')
                            else:
                                ret['professor_name'] = line[commaidx + 1:bar_idx - 1].replace(' / ', ', ')
                elif slashidx != -1 and slashidx < bar_idx - 7:    #/ ..
                    ret['class_name'] = line[idx + 4:slashidx - 4]
                    if line[slashidx + 2] != '.':
                        if line[slashidx + 5] == '/':
                            if line[slashidx + 9] in numbers:
                                ret['professor_name'] = line[slashidx - 4:slashidx - 1] + ', ' + line[slashidx + 2:slashidx + 4] + ', ' + line[slashidx + 7:bar_idx - 8]
                            else:
                                ret['professor_name'] = line[slashidx - 4:slashidx - 1] + ', ' + line[slashidx + 2:slashidx + 4] + ', ' + line[slashidx + 7:bar_idx - 8]
                        elif line[slashidx + 6] == '/':
                            if line[slashidx + 10] in numbers:
                                ret['professor_name'] = line[slashidx - 4:slashidx - 1] + ', ' + line[slashidx + 2:slashidx + 5] + ', ' + line[slashidx + 8:bar_idx - 8]
                            else:
                                ret['professor_name'] = line[slashidx - 4:slashidx - 1] + ', ' + line[slashidx + 2:slashidx + 5] + ', ' + line[slashidx + 8:bar_idx - 8]
                        else:
                            if line[slashidx + 4] in numbers:
                                ret['professor_name'] = line[slashidx - 4:slashidx - 1] + ', ' + line[slashidx + 2:bar_idx - 8]
                            elif line[slashidx + 5] in numbers:
                                ret['professor_name'] = line[slashidx - 4:slashidx - 1] + ', ' + line[slashidx + 2:bar_idx - 8]
                    else:
                        ret['professor_name'] = line[slashidx - 4:slashidx - 1]
                elif line[bar_idx - 7] in days:
                    ret['class_name'] = line[idx + 4:bar_idx - 11]
                    ret['professor_name'] = line[bar_idx - 11:bar_idx - 8]
                elif line[bar_idx - 1] in numbers or line[bar_idx - 1] in days:
                    ret['class_name'] = line[idx + 4:bar_idx - 4]
                    ret['professor_name'] = line[bar_idx - 4:bar_idx - 1]
                    
            else:
                ret['class_code'] = line[:idx + 3]
                if commaidx != -1 and commaidx < bar_idx:
                    if slashidx == -1 or slashidx >= bar_idx - 7:
                        ret['class_name'] = line[idx + 3:commaidx]
                        ret['professor_name'] = line[commaidx + 1:bar_idx - 8]  
                    elif slashidx < bar_idx - 7:
                        ret['class_name'] = line[idx + 3:commaidx]
                        if line[slashidx + 2] != '.':
                            ret['professor_name'] = line[commaidx + 1:bar_idx - 8].replace(' / ', ', ')
                elif slashidx != -1 and slashidx < bar_idx - 7:    #/ ..
                    ret['class_name'] = line[idx + 3:slashidx - 4]
                    if line[slashidx + 2] != '.':
                        if line[slashidx + 5] == '/':
                            if line[slashidx + 9] in numbers:
                                ret['professor_name'] = line[slashidx - 4:slashidx - 1] + ', ' + line[slashidx + 2:slashidx + 4] + ', ' + line[slashidx + 7:bar_idx - 8]
                            else:
                                ret['professor_name'] = line[slashidx - 4:slashidx - 1] + ', ' + line[slashidx + 2:slashidx + 4] + ', ' + line[slashidx + 7:bar_idx - 8]
                        elif line[slashidx + 6] == '/':
                            if line[slashidx + 10] in numbers:
                                ret['professor_name'] = line[slashidx - 4:slashidx - 1] + ', ' + line[slashidx + 2:slashidx + 5] + ', ' + line[slashidx + 8:bar_idx - 8]
                            else:
                                ret['professor_name'] = line[slashidx - 4:slashidx - 1] + ', ' + line[slashidx + 2:slashidx + 5] + ', ' + line[slashidx + 8:bar_idx - 8]
                        else:
                            if line[slashidx + 4] in numbers:
                                ret['professor_name'] = line[slashidx - 4:slashidx - 1] + ', ' + line[slashidx + 2:bar_idx - 8]
                            elif line[slashidx + 5] in numbers:
                                ret['professor_name'] = line[slashidx - 4:slashidx - 1] + ', ' + line[slashidx + 2:bar_idx - 8]
                    else:
                        ret['professor_name'] = line[slashidx - 4:slashidx - 1]
                elif line[bar_idx - 7] in days:
                    ret['class_name'] = line[idx + 3:bar_idx - 11]
                    ret['professor_name'] = line[bar_idx - 11:bar_idx - 8]
                elif line[bar_idx - 1] in numbers:
                    if line[bar_idx - 3:bar_idx - 1] == '..':
                        ret['class_name'] = line[idx + 3:bar_idx - 3]
                        ret['professor_name'] = np.NaN
                    else:
                        ret['class_name'] = line[idx + 3:bar_idx - 4]
                        ret['professor_name'] = line[bar_idx - 4:bar_idx - 1]
                elif line[bar_idx - 1] in days:
                    ret['class_name'] = line[idx + 3:bar_idx - 5]
                    ret['professor_name'] = line[bar_idx - 5:bar_idx-2]
        else:
            if line[idx - 7] in days:
                ret['date'].append(line[idx - 7])
                start_time, end_time = time_processing(line[idx - 5:idx + 6])
                ret['start_time'].append(start_time)
                ret['end_time'].append(end_time)
                
            elif line[idx - 1] in days:
                ret['date'].append(line[idx - 1])
                ret['start_time'].append(np.NaN)
                ret['end_time'].append(np.NaN)
    grade = []
                
    ret['class_code'], grade = class_code_processing(ret['class_code'])
    ret['grade'] = list_to_string(grade)
    ret['label'] = class_code_label(ret['class_code'])
    
    if len(ret['date']) == 0 and len(ret['start_time']) == 0:
        ret['start_time'].append(np.NaN)
        ret['end_time'].append(np.NaN)
    return ret

def string_to_list(string):
    ret = string.replace('[', '')
    ret = ret.replace(']', '')
    ret = ret.replace("'", '')
    
    ret = ret.split(', ')    
    return ret

def string_to_2dlist(string):
    ret = []
    temp = string.split('], [')
    for ele in temp:
        ret.append(string_to_list(ele))
    return ret

In [5]:
from IPython.display import display
processed_datas = []
keys = ['date', 'start_time', 'end_time', 'class_type', 'class_name', 'professor_name', 'grade', 'label']
df = pd.DataFrame(columns=keys)
for i, line in enumerate(data):
    processed_data = line_processing(line)
    for d, s_t, e_t in zip(processed_data['date'], processed_data['start_time'], processed_data['end_time']):
        df = df.append({
            'date' : d,
            'start_time' : s_t,
            'end_time' : e_t,
            'class_type' : processed_data['class_type'],
            'class_code' : processed_data['class_code'],
            'class_name' : processed_data['class_name'],
            'professor_name' : processed_data['professor_name'],
            'grade' : processed_data['grade'],
            'label' : processed_data['label']
        }, ignore_index=True)

In [6]:
label = list(df.drop_duplicates(['label'])['label'])
label = list(set(label))

In [7]:
final = pd.merge(left=df, right=major
                , how='left', on='label')

In [8]:
student = student[['student_name', 'student_id', 'major', 'period', 'grade', 'passed_subject', 'get_subject']]
student = student.drop_duplicates(['student_id'])

In [10]:
#df.to_csv('db_file.csv', encoding='utf-8-sig', index=None)
#final.to_csv('final_db.csv', encoding='utf-8-sig', index=None)

In [48]:
#label_df.to_csv('label_file.csv', encoding='utf-8-sig')
subject_time[subject_time[['start_time', 'end_time']].isnull()]

,class_code,date,start_time,end_time
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
3172,NaN,NaN,NaN,NaN
3173,NaN,NaN,NaN,NaN
3174,NaN,NaN,NaN,NaN
3175,NaN,NaN,NaN,NaN


In [41]:
final = final.drop_duplicates(final.keys())

In [42]:
subject_df = final[['class_code', 'class_name', 'professor_name']].drop_duplicates(['class_code'])
subject_time = final[['class_code', 'date', 'start_time', 'end_time']]

In [44]:
#subject_df.to_csv('subject_df.csv', encoding='utf-8-sig', index=None)
#subject_time.to_csv('subject_time.csv', encoding='utf-8-sig', index=None)
subject_time['class_code'][subject_time.duplicated(['class_code', 'date', 'start_time', 'end_time'])]
#subject_time.drop_duplicates(['class_code', 'date', 'start_time', 'end_time'])

Series([], Name: class_code, dtype: object)

In [9]:
student_df = pd.DataFrame(columns=['STUDENT_ID', 'STUDENT_NAME', 'MAJOR', 'PASSED_SEMESTER', 'GRADE'])
for i in range(len(student)):
    dic = dict(student.iloc[i])
    student_df = student_df.append({
        'STUDENT_ID' : str(dic['student_id']),
        'STUDENT_NAME' : dic['student_name'],
        'MAJOR' : dic['major'],
        'PASSED_SEMESTER' : dic['period'],
        'GRADE' : dic['grade']
    }, ignore_index=True)

In [10]:
student_df = student_df.drop_duplicates(['STUDENT_ID'])
student_df.to_csv('student_df.csv', encoding='utf-8-sig', index=None)

In [11]:
student_passed = pd.DataFrame(columns=['STUDENT_ID', 'PASSED_SUBJECT'])
student_get = pd.DataFrame(columns=['STUDENT_ID', 'GET_SUBJECT'])

In [12]:
for idx in range(len(student)):
    student_dic = dict(student.iloc[idx])
    passed_list = string_to_list(student_dic['passed_subject'])
    get_list = string_to_2dlist(student_dic['get_subject'])
    
    for p in passed_list:
        student_passed = student_passed.append({
            'STUDENT_ID' : str(student_dic['student_id']),
            'PASSED_SUBJECT' : p
        }, ignore_index=True)
        
    for g in get_list:
        student_get = student_get.append({
            'STUDENT_ID' : str(student_dic['student_id']),
            'GET_SUBJECT' : g[0],
            'CLASS_DAY' : g[3],
            'START_TIME' : g[1],
            'END_TIME' : g[2]
        }, ignore_index=True)

In [35]:
student_get[student_get['END_TIME'].isnull()]

,STUDENT_ID,GET_SUBJECT,CLASS_DAY,END_TIME,START_TIME


In [84]:
#student_passed.to_csv('student_passed.csv', encoding='utf-8-sig', index=None)
#student_get.to_csv('student_get.csv', encoding='utf-8-sig', index=None)

In [57]:
#subject_major = final[['class_code', 'major']]
#subject_major.to_csv('subject_major.csv', encoding='utf-8-sig', index=None)

# TIME TABLE

In [23]:
time = final.drop_duplicates(['date', 'start_time', 'end_time'])[['date', 'start_time', 'end_time']]

In [24]:
time

,date,start_time,end_time
0,월,09:00,10:15
1,수,09:00,10:15
2,화,09:00,10:15
3,목,09:00,10:15
4,화,12:00,13:15
...,...,...,...
3163,화,19:30,20:45
3168,목,18:00,20:45
3174,수,18:30,19:45
3175,수,20:00,21:15


In [54]:
subject_major = subject_major.drop_duplicates(['class_code', 'major'])
subject_major

,class_code,major
0,ELC204-00,"영미어문전공, 영미문화전공"
2,ELC204-01,"영미어문전공, 영미문화전공"
4,ELC203-00,"영미어문전공, 영미문화전공"
6,ELC203-01,"영미어문전공, 영미문화전공"
7,ELC202-00,"영미어문전공, 영미문화전공"
...,...,...
3171,IS3513-01,국제학과
3172,IE320-00,산업경영공학과
3173,NE446-00,원자력공학과
3174,EE368-04,전자과


In [61]:
subject_df

,class_code,class_name,professor_name
0,ELC204-00,Critical Reading and Writing 2,Todd Christopher Mcgowan
2,ELC204-01,Critical Reading and Writing 2,Todd Christopher Mcgowan
4,ELC203-00,Speech and Discussion 2,Peggy Cho
6,ELC203-01,Speech and Discussion 2,Peggy Cho
7,ELC202-00,문화연구입문,심효원
...,...,...,...
3171,IS3513-01,Capstone Design for International Studies,홍원택
3172,IE320-00,독립심화학습1(산업경영공학과),진창호
3173,NE446-00,독립심화학습1(원자력공학과),장윤석
3174,EE368-04,독립심화학습1(전자공학과),이승현


In [66]:
student_get = student_get[['STUDENT_ID', 'GET_SUBJECT']].drop_duplicates(['STUDENT_ID', 'GET_SUBJECT'])

In [77]:
student_list = list(student_df['STUDENT_ID'])

In [78]:
for student_id in student_get['STUDENT_ID']:
    if student_id not in student_list:
        print(student_id)

In [74]:
student_get['GET_SUBJECT']

0        GEE1127-G02
1        GED1508-G00
3        GED1329-G00
5        GEC1403-G18
7        GED1959-G02
            ...     
41045    GEC1403-G29
41047    GEC1105-G34
41049    GED1958-G00
41051    GEC1102-G14
41053    GEC1403-G34
Name: GET_SUBJECT, Length: 23933, dtype: object

In [83]:
student_passed = student_passed.drop_duplicates(['STUDENT_ID', 'PASSED_SUBJECT'])

In [81]:
student_passed

,STUDENT_ID,PASSED_SUBJECT
0,2019381529,GEC0103-G13
1,2019381529,APCH3501-00
2,2019381529,GEC0103-G11
3,2019381529,PM1016-01
4,2019381529,VID3009-00
...,...,...
59950,2021833183,AMTH1002-00
59951,2021833183,AMTH1002-14
59952,2021833183,CSE203-04
59953,2021833183,ME335-01
